In [1]:
import random,math,copy,time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier)
from sklearn import tree
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from tqdm import tqdm
import time
from sklearn.model_selection import GridSearchCV

C:\Users\SONAL BANSAL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
m=20  # Problem dimension (number of decision variables
Ap=0.1  # Awareness probability
fl=2   # Flight length (fl)
wf=1.2# wf is used to control the importance of classification accuracy and number of selected features.


In [11]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/707/original/Good_Job!.wav', autoplay=True))
## Insert whatever audio file you want above

In [3]:
df=pd.read_csv("AD-MCI.csv")
df = df.dropna(how='all')

In [22]:
df=df.drop(columns=['Unnamed: 0'])


In [28]:
df.head()


,label,0,1,2,3,4,5,6,7,8,...,67590,67591,67592,67593,67594,67595,67596,67597,67598,67599
0,0,55,56,57,57,57,57,56,55,56,...,59,59,59,59,59,59,60,59,58,57
1,0,255,255,255,255,255,255,254,253,254,...,255,255,255,255,255,255,255,255,255,255
2,0,255,255,255,255,255,255,254,253,254,...,255,255,255,255,255,255,255,255,255,255
3,0,255,255,255,255,255,255,254,253,254,...,255,255,255,255,255,255,255,255,255,255
4,0,255,255,255,255,255,255,254,253,254,...,255,255,255,255,255,255,255,255,255,255


In [13]:
val_Data = pd.read_csv("AD-MCI_val.csv")

In [14]:
val_Data = val_Data.dropna(how='all')
val_Data=val_Data.drop(columns=['Unnamed: 0'])
# df=df.reset_index().drop(columns='index')
print(val_Data.head())
allDone()

   label    0    1    2    3    4    5    6    7    8  ...    67590  67591  \
0      0  255  255  255  255  255  255  254  253  254  ...      255    255   
1      0  255  255  255  255  255  255  254  253  254  ...      255    255   
2      0  255  255  255  255  255  255  254  253  254  ...      255    255   
3      0  255  255  255  255  255  255  254  253  254  ...      255    255   
4      0  255  255  255  255  255  255  254  253  254  ...      255    255   

   67592  67593  67594  67595  67596  67597  67598  67599  
0    255    255    255    255    255    255    255    255  
1    255    255    255    255    255    255    255    255  
2    255    255    255    255    255    255    255    255  
3    255    255    255    255    255    255    255    255  
4    255    255    255    255    255    255    255    255  

[5 rows x 67601 columns]


In [19]:
features = [i for i in range(0,67600)]

In [24]:
d=67600# Flock (population) size
lt=67600
y_train=df['label']
x_train=df[df.columns[1:]]

In [25]:
y_test = val_Data['label']
x_test = val_Data[val_Data.columns[1:]]

In [11]:
# x=preprocessing.normalize(x)
#x=pd.DataFrame(x, columns=features)

In [26]:
x_train.columns=features
x_test.columns=features

In [13]:
imo=pd.read_csv('importance_feature_AD-MCI.csv').iloc[:,1]
len(imo)

67600

In [14]:
import time

In [46]:
def init():   #initialization function
    df2=pd.DataFrame(np.ones((m,d),dtype='int'),columns=features)
    return df2
#df2=init()

In [16]:
im=(rforest.feature_importances_)
print(im)
print(im.shape,type(im))

[0. 0. 0. ... 0. 0. 0.]
(67600,) <class 'numpy.ndarray'>


In [ ]:
im_df=pd.DataFrame(im)

In [ ]:
im_df.to_csv('importance.csv')

In [21]:
imo=pd.Series()
for i in tqdm(range(d)):
    imo= imo.set_value(features[i],im[i])

  0%|                                                                                        | 0/67600 [00:00<?, ?it/s]C:\Users\SONAL BANSAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
100%|████████████████████████████████████████████████████████████████████████████| 67600/67600 [20:17<00:00, 55.53it/s]


In [47]:
# this defines the fitness of each crow by calculating their accuracy and no features selected using the below formula
def fitness(df2):      # fitness function whose 1 parameter takes the crows and their position as input
    fnt=np.zeros((m,1))  # fnt is the fitness function
    for k in tqdm(range(m)):    
        global sel
        sel=[]
        q=0
        imp=0
        a=(df2.iloc[k,:])==0.0
        if a.all()==True:
            df2.iloc[k,:]=1.0
        for p in list(features):
            if df2.loc[k,p]==1.0:
                sel.insert(q,p)
                temp=imo[p]
                imp=imp+temp
            q=q+1
        lf=len(sel)
        divide=lf/lt
        fnt[k]=imp+wf*(1-float(lf/lt))     # fitness formula
    return fnt


In [ ]:
z=init()   #crow memory with initial position
tmax=50   # Maximum number of iterations (itermax)
xn=z.copy()  # the position of the crow
mem=init()  # memory is initialized using the init function
xnew=(xn).copy()   # the positions of the crow
ft=fitness(xn)
plotit=[]

In [50]:
tottime=0
tmax=50
#s=pd.DataFrame(z)
#print("got s")    # it is a formula to convert positions of each crow into 0 and 1 
for t in range(tmax):
    print(t)# no of iterations
    time1=time.time()
   
    xnew_list = []
    plotit.append(ft.max())
    num = random.randint(0,m-1)  
   # Generation of random candidate crows for following (chasing)
    for i in tqdm(range(m)):   
       
        r=random.random()
        if r>=Ap:
            
            xnew_list.append((xn.loc[i,:]+fl*r*(mem.loc[num,:]-xn.loc[i,:])).tolist())# Generation of a new position for crow i (state 1)
            
        else:
            
            xnew_list.append((xnew.loc[i,:].apply(lambda lam:random.random())).tolist())   # Generation of a new position for crow i (state 2)
            
            
    xnew = pd.DataFrame(xnew_list)
    del xnew_list
    print('transpose')
    xnew= 1/( 1 + np.expm1(( 10*(xnew-0.5))))    #  formula to convert positions of each crow into 0 and 1
    print('exp')
    xnew=xnew.apply(lambda l:l>=random.random()).astype(int)
    print('apply lambda')
    xn=xnew.copy()     
    print('copied')
    # it is the updated binary position of each crow
    #feasibility test
    # this is done so that to check if atleast one position of the crow has value equal to 1. if it is not true then every
    # position value of the crow is made 1.
    for i in tqdm(range(m)):
        
        if ((xn.loc[i,:])==0.0).all()==True:
            xn.loc[i,:]=1.0
    
    print('running fitness')
    ft_new=fitness(xn).reshape(ft.shape)# here the accuracy of each crows are calculated.

    print('fitness done')
    for i in tqdm(range(m)):
         
        if ft_new[i]>ft[i]:
            ft[i]=ft_new[i]
            for j in features:
                mem.loc[i,j]= xn.loc[i,j]
    
    print('complete'+str(t))
    time2=time.time()
    tottime=tottime+(time2-time1)
print(tottime/t+1)


0



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 54.03it/s]

transpose
exp
apply lambda
copied



100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 199.95it/s]

running fitness


...

running fitness



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:39<00:00,  4.98s/it]


fitness done



100%|██████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<?, ?it/s]

complete49
192.07184628564485


In [10]:
mem=pd.read_csv('memory.csv')

In [11]:
mem = mem.dropna(how='all')
mem=mem.drop(columns=['Unnamed: 0'])
mem.columns=features

# TESTING RESULTS

## Without Grid-Search, Without Bio-inspired

In [26]:
print(time.time())

neigh1 = KNeighborsClassifier()
neigh1.fit(x_train, y_train)
neigh1.predict(x_test)
nacc=neigh1.score(x_test,y_test)

print(nacc)

#
rforest1 = RandomForestClassifier()
rforest1.fit(x_train, y_train)
rforest1.predict(x_test)
rfacc=rforest1.score(x_test,y_test)

print(rfacc)

#
dtree1 = tree.DecisionTreeClassifier()
dtree1.fit(x_train, y_train)
dtree1.predict(x_test)
dtacc=dtree1.score(x_test,y_test)

print(dtacc)

clf1 = svm.SVC()
clf1.fit(x_train, y_train)
clf1.predict(x_test)
clfacc=clf1.score(x_test,y_test)

print(clfacc)



1553560919.5732362
1553561341.5669374
1553561347.9873168
1553561540.1038666
1553591247.9768016
0.7946666666666666 0.9173333333333333 0.8693333333333333 0.9386666666666666


## WITH GRID-SEARCH WITHOUT BIO-INSPIRED

In [21]:
dtree1 = tree.DecisionTreeClassifier()
# criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1,
dtparams = {'criterion':['gini', 'entropy'], 'splitter':['best','random'], 'max_depth':[30,40,50], 'min_samples_split':[10,20,30,40], 'min_samples_leaf':[4,6,8]}
grid_dt = GridSearchCV(dtree1, dtparams, verbose=2)
grid_dt.fit(x_train, y_train)
grid_dt.predict(x_test)
print(grid_dt.best_score_)
print(grid_dt.best_params_)
allDone()

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best 
[CV]  criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best, total= 1.2min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV] criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best 
[CV]  criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best, total= 1.0min
[CV] criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best 
[CV]  criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best, total= 1.1min
[CV] criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random 
[CV]  criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random, total=  15.4s
[CV] criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random 
[CV]  criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random, total=  13.0s
[CV] criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random 
[CV]  criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random, total=  14.4s
[CV

[CV] criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random 
[CV]  criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random, total=  18.1s
[CV] criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random 
[CV]  criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random, total=  18.8s
[CV] criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=20, splitter=best 
[CV]  criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=20, splitter=best, total= 1.4min
[CV] criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=20, splitter=best 
[CV]  criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=20, splitter=best, total= 1.3min
[CV] criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=20, splitter=best 
[CV]  criterion=gini, max_depth=20, min_samples_leaf=4, min_samples_split=20, splitter=best, total= 1.2m

[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best, total= 1.1min
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=best, total= 1.1min
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random, total=  14.0s
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random, total=  12.3s
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, splitter=random 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=5, spli

[CV] criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=best 
[CV]  criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=best, total= 1.1min
[CV] criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random 
[CV]  criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random, total=  17.8s
[CV] criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random 
[CV]  criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random, total=  15.7s
[CV] criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random 
[CV]  criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=10, splitter=random, total=  15.3s
[CV] criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=20, splitter=best 
[CV]  criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_sp

[CV] criterion=entropy, max_depth=30, min_samples_leaf=6, min_samples_split=20, splitter=random 
[CV]  criterion=entropy, max_depth=30, min_samples_leaf=6, min_samples_split=20, splitter=random, total=  13.3s
[CV] criterion=entropy, max_depth=30, min_samples_leaf=6, min_samples_split=20, splitter=random 
[CV]  criterion=entropy, max_depth=30, min_samples_leaf=6, min_samples_split=20, splitter=random, total=  12.4s
[CV] criterion=entropy, max_depth=30, min_samples_leaf=6, min_samples_split=20, splitter=random 
[CV]  criterion=entropy, max_depth=30, min_samples_leaf=6, min_samples_split=20, splitter=random, total=  11.4s


[Parallel(n_jobs=1)]: Done 324 out of 324 | elapsed: 227.9min finished


0.6425087108013937
{'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'splitter': 'random'}


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
rforest1 = RandomForestClassifier()
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 800, num = 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
rfparams = {'criterion':['gini', 'entropy'],
            'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

grid_rf = GridSearchCV(estimator = rforest1, param_grid= rfparams,  cv = 3, verbose=2,  n_jobs = -1)
allDone()

grid_rf.fit(x_train, y_train)
allDone()
print(grid_rf.grid_scores_)
print(grid_rf.best_score_)
print(grid_rf.best_params_)

allDone()

grid_rf.predict(x_test)
gridacc=grid_rf.score(x_test,y_test)


allDone()

Fitting 3 folds for each of 39600 candidates, totalling 118800 fits


In [32]:
grid_rf.score()

<bound method BaseSearchCV.score of GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)>

## Without Grid-Seach ,With Bio-Inspired

In [24]:
# here the accuracy of each crows are calculated.
neigh = KNeighborsClassifier()
rforest =RandomForestClassifier()
dtree = tree.DecisionTreeClassifier()
clf = svm.SVC()

nacc1=np.zeros((m,1),dtype='float64')
rfacc1=np.zeros((m,1),dtype='float64')
dtacc1=np.zeros((m,1),dtype='float64')
clfacc1=np.zeros((m,1),dtype='float64')

q=0
for i in tqdm(range(m)):
    print(i)
    select=[]
    for p in list(features):
        if mem.loc[i,p]==1:
            select.insert(q,p)
            q=q+1
    neigh.fit(x_train[select], y_train)
    neigh.predict(x_test[select])
    nacc1[i]=neigh.score(x_test[select],y_test) 
    print(nacc1[i])
    
    rforest.fit(x_train[select], y_train)
    rforest.predict(x_test[select])
    rfacc1[i]=rforest.score(x_test[select],y_test) 
    print(rfacc1[i])
    
    dtree.fit(x_train[select], y_train)
    dtree.predict(x_test[select])
    dtacc1[i]=dtree.score(x_test[select],y_test)
    print(dtacc1[i])
    
    clf.fit(x_train[select], y_train)
    clf.predict(x_test[select])
    clfacc1[i]=clf.score(x_test[select],y_test)
    print(clfacc1[i])

print (nacc1.max(), rfacc1.max(), dtacc1.max(),clfacc1.max()) # the crow with maximum accuracy is printed


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

0
[0.784]
[0.90133333]
[0.864]
[0.93866667]



  5%|████                                                                            | 1/20 [06:00<1:54:07, 360.38s/it]

1
[0.76266667]
[0.904]
[0.864]
[0.93866667]



 10%|████████▏                                                                         | 2/20 [06:14<56:09, 187.20s/it]

...

18
[0.76266667]
[0.904]
[0.85333333]
[0.93866667]



 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [09:46<00:30, 30.86s/it]

19
[0.76266667]
[0.89333333]
[0.84266667]
[0.93866667]



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [09:56<00:00, 29.83s/it]


NameError: name 'clf1' is not defined

In [25]:
print (nacc1.max(), rfacc1.max(), dtacc1.max(),clfacc1.max()) # the crow with maximum accuracy is printed

0.784 0.9146666666666666 0.8693333333333333 0.9386666666666666


**NUMBER OF FEATURES SELECTED**

In [28]:
print ((mem.loc[nacc1.argmax(),:]==1.0).sum())

print ((mem.loc[rfacc1.argmax(),:]==1.0).sum())
print ((mem.loc[dtacc1.argmax(),:]==1.0).sum())
print ((mem.loc[clfacc1.argmax(),:]==1.0).sum())

14368
469
469
14368


**No. of Iterations  VS Fitness Value**

In [30]:
plotse=pd.Series(plotit)
plt.plot(plotse)
plt.xlabel('No of iterations')
plt.ylabel('Fitness value')
plt.show()

NameError: name 'plotit' is not defined

In [75]:
mem.to_csv('memory.csv')